<h1 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>Melbourne Backpack's student accommodation Star Rating System</strong></h1>

| Rating 	| Interpretation  	|
|---	|---	|
| ⭐  	| A one-star rating often means a property has no frills and only offers basic accommodations. A one-star rating doesn’t necessarily mean an accomodation is dirty, unkept or in a bad location. It simply means you’ll have a place to sleep, and that’s pretty much it.  	|
| ⭐⭐  	| Like one-star, two-star properties are typically more affordable than student living that have a higher rating — they are also usually quite comfortable.  	|
| ⭐⭐⭐  	| Three-star student living ordinarily have some unique amenities and provide quality service.  	|
| ⭐⭐⭐⭐  	| Four-star student living are often noted for their upscale quality and extraordinary comfort.  	|
| ⭐⭐⭐⭐⭐  	| A five-star property provides flawless guest services in a state-of-the-art facility. As a five-star property, such as premium dining options and personalized services to its students. With no detail being overlooked, these student living commonly even provides high-end, luxury toiletries for students.  	|

Next, let's have a look at the review. First, noticeable parts are the html strings like \ r \ n, and the parts that express emotions in parentheses such as (very unusual for him) and (a good thing) and words in capital letters like MUCH.

In [1]:
!pip install ktrain
import pandas as pd
import numpy as np
import ktrain
import tensorflow as tf
from ktrain import text

In [2]:
tf.version.VERSION

'2.8.0'

In [69]:
df_train = pd.read_csv('../data/interim/reviews/external_reviews.csv', encoding='utf-8')
df_test = pd.read_csv('../data/interim/reviews/dwell.csv', encoding='utf-8')

df_train

,comment,rating,link,category_id
0,"Friendly staff, convenient location near Victo...",5,https://www.facebook.com/unilodge.vu/reviews,4
1,I stay here for 2 year and it make me feel hom...,5,https://www.facebook.com/unilodge.vu/reviews,4
2,I makes a lot of friend here too and having a ...,5,https://www.facebook.com/unilodge.vu/reviews,4
3,The staff is very polite and gentle specially ...,5,https://www.facebook.com/unilodge.vu/reviews,4
4,5 Star service experience here at UniLodge @ V...,5,https://www.facebook.com/unilodge.vu/reviews,4
...,...,...,...,...
94,Overall good. I will defo come...Movie room an...,4,https://planetofhotels.com/en/australia/melbou...,1
95,It was excellent overall and...The pool and th...,5,https://planetofhotels.com/en/australia/melbou...,1
96,Good,5,https://planetofhotels.com/en/australia/melbou...,1
97,Very convenient. Great location. Ease of check...,5,https://planetofhotels.com/en/australia/melbou...,1


In [70]:
df_train.rating.value_counts()

5    45
1    22
4    12
3    11
2     9
Name: rating, dtype: int64

In [71]:
X_train = df_train.comment.tolist()
X_test = df_test.comment.tolist()
y_train = df_train.rating.tolist()
y_test = df_test.rating.tolist()

In [72]:
# Checking size of our train and test datasets:
print(len(X_train),len(X_test),len(y_train),len(y_test))

99 23 99 23


In [75]:
stars = [1, 2, 3, 4, 5]

In [76]:
# encoding = { '1': 0,
#     '2': 1,
#     '3': 2,
#     '4': 3,
#     '5': 4
# }

# y_train = [encoding[key] for key in y_train]
# y_test = [encoding[key] for key in y_test]

# Building Model using Transformer

We are using bert-base-uncased model. You can choose any other model. I am selecting maxlen of tokenization as 512 (it's max for BERT).

In [77]:
model_arch ='bert-base-uncased'
MAXLEN = 512
trans = text.Transformer(model_arch, maxlen=MAXLEN, class_names= stars)


In [78]:
print(y_test)

[5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 5, 4, 2, 5, 5, 4, 5, 4, 4, 5, 5, 5, 5]


Let's preprocess out test and train data set.

In [82]:
train_data = trans.preprocess_train(X_train,y_train)
test_data = trans.preprocess_test(X_test,y_test)

In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=10)

Finding the best learning rate:

In [83]:
learner.lr_find(show_plot=True, max_epochs=10)

In [84]:
learner.fit_onecycle(3e-5, 5)

**Confusion Matrix:**

In [ ]:
learner.validate(val_data=test_data, class_names=stars)

**top 5 data points not performing good:**

In [ ]:
learner.view_top_losses(n=5, preproc=trans)

In [ ]:
X_test[1928]

Above data our model is predicting as sadness but label is mentioned as fear.

# Predict Data:

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [ ]:
inp = 'I am very disappointed with this kind of accomodation. Need refund.'

In [ ]:
predictor.predict(inp)